In [62]:
import re
import pickle
import numpy as np
import pandas as pd

In [2]:
from protmapper import ProtMapper
from protmapper.uniprot_client import get_gene_name
from protmapper.uniprot_client import get_id_from_mnemonic

# Processing EGFR Data so we can use INDRA Network

### Get Uniprot ID's, Gene Symbols, Sequences, and Sites out of Peptide Map

In [3]:
peptide_map = pd.read_csv('/Users/albertsteppi/tps/data/timeseries/peptide-mapping.tsv',
                             sep='\t')
peptide_map.columns = ['peptide','mnemonic']

#### Uniprot IDs and Gene Symbols
The data contains Uniprot Mnemonics which we need to map to id's

In [4]:
peptide_map['Uniprot_Id'] = peptide_map.mnemonic.apply(get_id_from_mnemonic)

Some of the Uniprot mnemonics are outdated. We cannot recover the current Uniprot ID from
these so we throw them away.

In [5]:
peptide_map = peptide_map[~peptide_map.Uniprot_Id.isnull()]

We can then get Gene Symbols for these Uniprot Ids. 

In [6]:
peptide_map['Gene_Symbol'] = peptide_map.Uniprot_Id.apply(get_gene_name)

We remove the entries where we were unable to find a gene symbol for the given
Uniprot Id. (Some proteins included are unreviewed, speculative, or now obsolete.)

In [7]:
peptide_map = peptide_map[~peptide_map.Gene_Symbol.isnull()].copy()

#### Peptides
Peptides in the data are encoded by strings of the form `K.n[305.21]AELEEMEEVHPS[167.00]DEEEEDATK[432.30].A`
The Capital letters besides those that start and end the sequence
are residues. Serines (S) followed by `[167.00]` are phosporylated,
as are Threonines (T) followed by `[181.01]` and Tyrosines (Y) followed
by `[243.03]`. Given such an encoding, we use the following functions to 
separate out the sequence and get the indices of the sites in this sequence,
indexed beginning at 1.

In [8]:
def get_sequence(peptide):   
    pep = peptide[11:-2]
    return re.sub(r'[[0-9]*\.[0-9]*\]', '', pep)


def get_local_sites(peptide):
    sequence = get_sequence(peptide)
    groups = re.finditer(r'[[0-9]*\.[0-9]*\]', peptide[11:-2])
    results = []
    for index, group in enumerate(groups):
        position = group.span()[0] - 8*index
        residue = sequence[position-1]
        if residue in ['T', 'S', 'Y']:
            results.append((residue, position))
    return results

In [9]:
peptide_map['Motif'] = peptide_map.peptide.apply(get_sequence)

/Users/albertsteppi/.virtualenvs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Possible nested set at position 1
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
peptide_map['local_sites'] = peptide_map.peptide.apply(get_local_sites)

We can use the ProtMapper to get the global site positions based on the uniprot id's, sequences, and 1-indexed positions within the sequences.

In [11]:
def get_site_positions(uniprot_id, sequence, positions):
    pm = ProtMapper()
    result = []
    for position in positions:
        mapped_site = pm.map_peptide_to_human_ref(uniprot_id, 'uniprot', sequence, position)
        result.append(mapped_site.mapped_pos)
    return ';'.join([x for x in result if x])

In [12]:
peptide_map['Site_Position'] = peptide_map.apply(lambda row:
                                                        get_site_positions(row.Uniprot_Id,
                                                        row.Motif,
                                                        [pos for site, pos
                                                         in row.local_sites]), axis=1)

INFO: [2019-08-27 14:08:13] protmapper.uniprot_client - Loading Swissprot sequences...
INFO: [2019-08-27 14:08:15] protmapper.uniprot_client - Loading Uniprot isoform sequences...


A small number of sites could not be identified by the ProtMapper. We exclude these

In [13]:
peptide_map = peptide_map[~peptide_map.Site_Position.apply(lambda x: not x)]

In [14]:
peptide_mapping = peptide_map[['peptide', 'Gene_Symbol']]
peptide_mapping.columns = ['peptide', 'protein(s)']
peptide_mapping.to_csv('../work/EGFR/peptide_mapping.tsv', sep='\t', index=False)

## Time series files

We get rid of the entries in the time series files corresponding to peptides that were
filtered out in the above process

In [15]:
timecourse = pd.read_csv('/Users/albertsteppi/tps/data/timeseries/median-time-series.tsv',
                         sep='\t')

In [17]:
timecourse = timecourse[timecourse.peptide.isin(peptide_mapping.peptide)]
timecourse.to_csv('../work/EGFR/median-time-series.tsv', sep='\t', index=False)

In [22]:
p_first = pd.read_csv('/Users/albertsteppi/tps/data/timeseries/p-values-first.tsv',
                      sep='\t')
p_prev = pd.read_csv('/Users/albertsteppi/tps/data/timeseries/p-values-prev.tsv',
                     sep='\t')

In [25]:
p_first = p_first[p_first['#peptide'].isin(peptide_mapping.peptide)]
p_prev = p_prev[p_prev['#peptide'].isin(peptide_mapping.peptide)]

In [28]:
p_first.to_csv('../work/EGFR/p_values_first.tsv', sep='\t', index=False)
p_prev.to_csv('../work/EGFR/p_values_prev.tsv', sep='\t', index=False)

## Protein Prizes

We must now calculate the protein prizes associated to each protein. These are needed to
run the steiner analysis and were not included in the data in the TPS repo

In [50]:
p_values = pd.merge(p_first, p_prev.drop('2min.vs.0min', axis=1), on='#peptide')
p_values.rename(columns={'#peptide': 'peptide'}, inplace=True)

In [51]:
p_values['min'] = p_values.iloc[:, 1:].apply(min, axis=1)

In [53]:
p_values = pd.merge(p_values, peptide_mapping, on='peptide')

In [55]:
prizes = -np.log10(p_values.groupby('protein(s)')['min'].min())

In [57]:
prizes.sort_values(ascending=False, inplace=True)

In [59]:
prizes = pd.DataFrame(prizes).reset_index()

In [60]:
prizes.columns = ['name', 'prize']

Following Koksal et al. we keep only proteins with significant prizes. The following keeps only proteins with a minumum p-value among all peptides and considered time differences of 0.01 or less. This is the same as was done in the original paper.

In [113]:
prizes = prizes[prizes.prize >= 2.0]

In [115]:
prizes.to_csv('../work/EGFR/protein_prizes.tsv', sep='\t', header=True, index=False)

## Prior
We take a collection of all phosphorylations in the INDRA database from phosphosite plus
and all phosphorylations and dephosphorylations from signor. These are used to form a prior
network of edges with known direction and sign. Below we process the json's of these statements into a prior network as taken by TPS.

In [63]:
with open('../input/phosphorylation.pkl', 'rb') as f:
    phospho_statements = pickle.load(f)

In [94]:
activations = set()
inhibitions = set()
for stmt in phospho_statements:
    enzyme = stmt['enz']
    substrate = stmt['sub']
    if 'HGNC' not in enzyme['db_refs'] or 'HGNC'not in substrate['db_refs']:
        continue
    enz_name = enzyme['name']
    sub_name = substrate['name']
    if stmt['type'] == 'Phosphorylation':
        activations.add((enz_name, sub_name))
    else:
        inhibitions.add((enz_name, sub_name))

These statements are site specific. For this cvase we flatten to general phosphorylations independend of site. There is one case where one protein is listed as both phosphorylating and dephosphorylating another. We exclude this as an edge in the prior because TPS cannot
handle such situations.

In [95]:
activations & inhibitions

{('PPP2CA', 'FOXO3')}

In [96]:
activations = activations - inhibitions
inhibitions = inhibitions - activations

In [97]:
records = [[a[0], 'A', a[1]] for a in activations]
records.extend([[i[0], 'I', i[1]] for i in inhibitions])

In [98]:
prior = pd.DataFrame.from_records(records)

In [100]:
prior.to_csv('../work/EGFR/kinase_substrate.sif', sep='\t', index=False, header=False)